# Notebook 01 - Importação e Limpeza de Dados

#### Inicialmente, iremos importar o dataset .csv para um dataframe Pandas e depois salvar os dados no banco de dados PostgreSQL, para simular um cenário realista de bancos, os quais guardam os dados de seus clientes em bancos de dados relacionais.

1. Importação das bibliotecas necessárias e leitura do dataset .csv em um dataframe Pandas.

In [68]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

data = pd.read_csv('../data/german_credit_data.csv')
data.head()


,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


In [69]:
print(data.columns)

Index(['Unnamed: 0', 'Age', 'Sex', 'Job', 'Housing', 'Saving accounts',
       'Checking account', 'Credit amount', 'Duration', 'Purpose', 'Risk'],
      dtype='object')


2. Criamos uma função para estimar o salário anual com base no nível do trabalho (Job Level) e aplicamos essa função para criar uma nova coluna no dataframe.

In [70]:
import numpy as np

def estimar_salario(job_level):
    if job_level == 0:
        return np.random.uniform(15000, 22000) # R$ anual
    elif job_level == 1:
        return np.random.uniform(25000, 40000)
    elif job_level == 2:
        return np.random.uniform(45000, 80000)
    elif job_level == 3:
        return np.random.uniform(90000, 180000)
    else:
        return np.random.uniform(30000, 50000)

In [71]:
data['salario_anual'] = data['Job'].apply(estimar_salario) # Aplicar a função estimar_salario numa coluna nova 'salario_anual' usando 'Job' como referência
data['salario_anual'] = data['salario_anual'].round(2) # Arredonda para 2 casas decimais para deixar semelhante ao dinheiro real

data = data.drop(columns=['Unnamed: 0', 'Job'], axis=1) # Remove colunas desnecessárias

dicionario = { # Cria um dicionário para poder renomear as colunas com base no nome das colunas das tabelas do banco de dados
    'Age': 'idade',
    'Sex': 'sexo',
    'Housing': 'situacao_moradia',
    'Saving accounts': 'status_poupanca',
    'Checking account': 'status_conta_corrente',
    'Credit amount': 'valor_emprestimo',
    'Duration': 'prazo_meses',
    'Purpose': 'finalidade_emprestimo',
    'Risk': 'status_inadimplencia'
}

data = data.rename(columns=dicionario) # Renomeia as colunas do dataframe

data['status_inadimplencia'] = data['status_inadimplencia'].map({'good': 0, 'bad': 1}) # converte valores good e bad para 0 e 1
data.head(10)

,idade,sexo,situacao_moradia,status_poupanca,status_conta_corrente,valor_emprestimo,prazo_meses,finalidade_emprestimo,status_inadimplencia,salario_anual
0,67,male,own,NaN,little,1169,6,radio/TV,0,46281.64
1,22,female,own,little,moderate,5951,48,radio/TV,1,56886.66
2,49,male,own,little,NaN,2096,12,education,0,36356.54
3,45,male,free,little,little,7882,42,furniture/equipment,0,72999.24
4,53,male,free,little,little,4870,24,car,1,56517.81
5,35,male,free,NaN,NaN,9055,36,education,0,29579.25
6,53,male,own,quite rich,NaN,2835,24,furniture/equipment,0,69989.37
7,35,male,rent,little,moderate,6948,36,car,0,98879.62
8,61,male,own,rich,NaN,3059,12,radio/TV,0,26033.42
9,28,male,own,little,moderate,5234,30,car,1,139507.77


3. Assim como foi feito para a coluna de salário anual, iremos mudar as colunas status_poupança e status_conta_corrente para valores numéricos. Para começar, precisamos tratar os valores nulos nessas colunas.

In [72]:
data['status_conta_corrente'] = data['status_conta_corrente'].fillna('unknown') # preenche valores nulos com 'unknown' para evitar erros na conversão
data['status_poupanca'] = data['status_poupanca'].fillna('unknown') # preenche valores nulos com 'unknown' para evitar erros na conversão

4. Em seguida, definimos as funções para estimar os valores das contas corrente e poupança com base nos níveis categóricos.

In [73]:
data['status_conta_corrente'] = data['status_conta_corrente'].map({'little': 0, 'moderate': 1, 'rich': 2, 'quite rich': 3}) # repete o processo feito para o salário anual
data['status_poupanca'] = data['status_poupanca'].map({'little': 0, 'moderate': 1, 'rich': 2, 'quite rich': 3})

def estimar_conta_corrente(conta_corrente):
    if conta_corrente == 0:
        return np.random.uniform(25000, 40000)
    elif conta_corrente == 1:
        return np.random.uniform(45000, 80000)
    elif conta_corrente == 2:
        return np.random.uniform(90000, 180000)
    elif conta_corrente == 3:
        return np.random.uniform(100001, 1000000)
    else:
        return 0 # retorna None por enquanto

def estimar_conta_poupanca(conta_poupanca):
    if conta_poupanca == 0:
        return np.random.uniform(1, 1000)
    elif conta_poupanca == 1:
        return np.random.uniform(1001, 10000)
    elif conta_poupanca == 2:
        return np.random.uniform(10001, 100000)
    elif conta_poupanca == 3:
        return np.random.uniform(100001, 1000000)
    else:
        return 0

5. Aplicando a função nas colunas

In [74]:
data['status_conta_corrente'] = data['status_conta_corrente'].apply(estimar_conta_corrente).round(2)
data['status_poupanca'] = data['status_poupanca'].apply(estimar_conta_poupanca).round(2)

6. Renomeando as colunas para refletir os novos valores

In [75]:
dicionario_colunas = {
    'status_conta_corrente': 'valor_conta_corrente',
    'status_poupanca': 'valor_conta_poupanca'
}

data = data.rename(columns=dicionario_colunas)

data.head(10)

,idade,sexo,situacao_moradia,valor_conta_poupanca,valor_conta_corrente,valor_emprestimo,prazo_meses,finalidade_emprestimo,status_inadimplencia,salario_anual
0,67,male,own,0.00,27164.52,1169,6,radio/TV,0,46281.64
1,22,female,own,613.78,48999.87,5951,48,radio/TV,1,56886.66
2,49,male,own,52.37,0.00,2096,12,education,0,36356.54
3,45,male,free,982.68,37072.32,7882,42,furniture/equipment,0,72999.24
4,53,male,free,305.36,33075.00,4870,24,car,1,56517.81
5,35,male,free,0.00,0.00,9055,36,education,0,29579.25
6,53,male,own,117699.55,0.00,2835,24,furniture/equipment,0,69989.37
7,35,male,rent,328.72,45413.27,6948,36,car,0,98879.62
8,61,male,own,89884.20,0.00,3059,12,radio/TV,0,26033.42
9,28,male,own,868.28,76816.96,5234,30,car,1,139507.77
